In [4]:
import numpy as np
import so101
import importlib

Test the implementation of so101

In [6]:
importlib.reload(so101)
# Compute forward kinematics from saved angles and compare to saved poses
data = np.load('fk_test_dataset.npz', allow_pickle=True)

angles = data['angles']
poses_saved = data['poses']
joint_names = data['joint_names'].tolist()
n = angles.shape[0]

computed_poses = np.zeros_like(poses_saved)
for i in range(n):
    theta = {name: float(angles[i, idx]) for idx, name in enumerate(joint_names)}
    computed_poses[i] = so101.SO101.forward_kinematics(theta)

# Evaluate errors
# These evaluations are pretty niave as they should probably be computed in the tangent space and not use the euclidean difference
errors = np.abs(computed_poses - poses_saved)
max_err = errors.max()
mean_err = errors.mean()
print(f'max error: {max_err:.12e}, mean error: {mean_err:.12e}')
if max_err < 1e-9:
    print('Forward kinematics test PASSED — recomputed poses match saved poses')
else:
    idxs = np.where(np.max(errors.reshape(n, -1), axis=1) > 1e-9)[0]
    print(f'FAILED on {len(idxs)} samples, sample indices: {idxs}')
    if len(idxs) > 0:
        k = idxs[0]
        print('saved pose:\n', poses_saved[k])
        print('recomputed pose:\n', computed_poses[k])
        print('diff:\n', computed_poses[k] - poses_saved[k])

max error: 0.000000000000e+00, mean error: 0.000000000000e+00
Forward kinematics test PASSED — recomputed poses match saved poses


Used to generate the testing dataset
Requires a confirmed working version of so101

In [3]:
# Generate a testing dataset
# Note: SO101.forward_kinematics expects a dict of joint_name: angle_in_degrees
# and returns a 4x4 transformation matrix.
# Joint names (ordered as in the robot model)
joint_names = list(so101.SO101.joint_twists.keys())
n = 30
rng = np.random.default_rng(seed=42)
# generate angles in degrees in range [-180, 180]
angles = rng.uniform(-180, 180, size=(n, len(joint_names)))

# Compute FK for each sample and store as 4x4 matrices
poses = np.zeros((n, 4, 4))
for i in range(n):
    ja = {name: float(angles[i, idx]) for idx, name in enumerate(joint_names)}
    poses[i] = so101.SO101.forward_kinematics(ja)

# Save to compressed numpy file in the notebook folder
save_path = 'fk_test_dataset.npz'
np.savez_compressed(save_path, angles=angles, poses=poses, joint_names=np.array(joint_names, dtype=object))
print(f'Saved {n} samples to {save_path}')

Saved 30 samples to fk_test_dataset.npz


In [4]:
# Load the saved dataset and inspect
import numpy as np
data = np.load('fk_test_dataset.npz', allow_pickle=True)
angles = data['angles']
poses = data['poses']
joint_names = data['joint_names'].tolist()
print('angles shape', angles.shape)
print('poses shape', poses.shape)
print('joint names', joint_names)
print('first angles:', angles[0])
print('first pose:\n', poses[0])

angles shape (30, 5)
poses shape (30, 4, 4)
joint names ['shoulder_pan', 'shoulder_lift', 'elbow_flex', 'wrist_flex', 'wrist_roll']
first angles: [  98.62417748  -22.00376169  129.09525117   71.05249046 -146.09615476]
first pose:
 [[ 0.14987389 -1.37076249  0.54746296  0.06730788]
 [ 0.9881745   0.21681602 -0.11022094  0.18773034]
 [-0.032388    0.27203125 -0.82953942  0.09719081]
 [ 0.          0.          0.          1.        ]]
